In [1]:
import os

import sys
sys.path.append("../project_functions/")
import ad_functions as adfns
import ad_crnn_functions as adcrn


import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import (
    Input, 
    Conv2D, 
    MaxPooling2D, 
    Reshape, 
    Bidirectional, 
    LSTM, 
    Dense, 
    BatchNormalization, 
    Dropout, 
    Softmax, 
    Flatten
)

In [2]:
data = pd.read_csv('digit_training_data.csv')

In [4]:
# def load_data():
#     # Load the CSV file into a Pandas DataFrame
#     df = pd.read_csv("digit_training_data.csv")
#     # Extract the file names and digit labels from the DataFrame
#     file_names = df["file_name"].values
#     labels = df["digit"].values
#     # Load the digit images and preprocess them
#     images = []
#     for file_name in file_names:
#         image = cv2.imread(os.path.join("./digits", file_name), cv2.IMREAD_GRAYSCALE)
#         image = cv2.resize(image, (32, 32))
#         image = np.expand_dims(image, axis=-1)
#         images.append(image)
#     images = np.array(images)
#     # Convert the digit labels to one-hot encoding
#     num_classes = 10
#     one_hot_labels = np.zeros((len(labels), num_classes))
#     for i, label in enumerate(labels):
#         one_hot_labels[i, int(label)] = 1
#     # Return the images and one-hot labels
#     return images, one_hot_labels

In [13]:
# def load_data():
#     # Load the CSV file into a Pandas DataFrame
#     df = pd.read_csv("digit_training_data.csv")
#     # Extract the file names and digit labels from the DataFrame
#     file_names = df["file_name"].values
#     labels = df["digit"].values
#     # Load the digit images and preprocess them
#     images = []
#     for file_name in file_names:
#         image = cv2.imread(os.path.join("./digits", file_name))
#         image = cv2.resize(image, (32, 32))
#         images.append(image)
#     images = np.array(images)
#     # Convert the digit labels to one-hot encoding
#     num_classes = 10
#     one_hot_labels = np.zeros((len(labels), num_classes))
#     for i, label in enumerate(labels):
#         one_hot_labels[i, int(label)] = 1
#     # Return the images and one-hot labels
#     return images, one_hot_labels


In [18]:
def one_hot_encode_labels(labels, num_digits, num_classes):
    # Initialize an array of zeros to hold the one-hot encoded labels
    one_hot_labels = np.zeros((len(labels), num_digits, num_classes))
    # Loop over the labels and one-hot encode each digit separately
    for i, label in enumerate(labels):
        for j, digit in enumerate(str(label)):
            one_hot_labels[i, j, int(digit)] = 1
    return one_hot_labels


In [19]:
def load_data():
    # Load the CSV file into a Pandas DataFrame
    df = pd.read_csv("digit_training_data.csv")
    # Extract the file names and digit labels from the DataFrame
    file_names = df["file_name"].values
    labels = df["digit"].values
    # One-hot encode the labels for each digit separately
    num_digits = 5
    num_classes = 10
    one_hot_labels = one_hot_encode_labels(labels, num_digits, num_classes)
    # Load the digit images and preprocess them
    images = []
    for file_name in file_names:
        image = cv2.imread(os.path.join("./digits", file_name))
        image = cv2.resize(image, (32, 32))
        images.append(image)
    images = np.array(images)
    # Return the images and one-hot labels
    return images, one_hot_labels


In [20]:
# Load the digit images and labels
images, labels = load_data()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [15]:
def create_crnn_model(
    input_shape=(32, 32, 3), num_digits=5, num_classes=11, learning_rate=1e-4
):
    input_layer = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(512, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(num_classes * num_digits, activation="linear")(x)
    x = Reshape((num_digits, num_classes))(x)
    x = Softmax(axis=-1)(x)

    model = Model(inputs=input_layer, outputs=x)
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


In [21]:
model = create_crnn_model(input_shape=(32, 32, 3), num_digits=5, num_classes=10, learning_rate=1e-4)
model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])


In [22]:
model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    batch_size=64,
    epochs=25,
    verbose=1
)


Epoch 1/25
2500/2500 [==============================] - 17s 6ms/step - loss: 0.3834 - accuracy: 0.1659 - val_loss: 0.1991 - val_accuracy: 0.2119
Epoch 2/25
2500/2500 [==============================] - 14s 6ms/step - loss: 0.2499 - accuracy: 0.2037 - val_loss: 0.1722 - val_accuracy: 0.2367
Epoch 3/25
2500/2500 [==============================] - 14s 6ms/step - loss: 0.2718 - accuracy: 0.1910 - val_loss: 0.1939 - val_accuracy: 0.2159
Epoch 4/25
2500/2500 [==============================] - 14s 6ms/step - loss: 0.3023 - accuracy: 0.1797 - val_loss: 0.2007 - val_accuracy: 0.2072
Epoch 5/25
2500/2500 [==============================] - 14s 6ms/step - loss: 0.3171 - accuracy: 0.1729 - val_loss: 0.2090 - val_accuracy: 0.2376
Epoch 6/25
2500/2500 [==============================] - 14s 6ms/step - loss: 0.3404 - accuracy: 0.1636 - val_loss: 0.2581 - val_accuracy: 0.1815
Epoch 7/25
2500/2500 [==============================] - 14s 6ms/step - loss: 0.3800 - accuracy: 0.1489 - val_loss: 0.2915 - val_ac